In [9]:
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
from tqdm import tqdm

In [10]:
def run_rashomon_simulation(thresholds=[0.01], 
                           sample_sizes=[50, 1000, 5000, 10000],
                           n_iterations=100,
                           n_total=100000,
                           n_features=30,
                           k=5,  # match the "true" model complexity
                           noise_level=1.0,
                           save_results=True,
                           random_seed=42):
    """
    Run the Rashomon effect simulation where true DGP is a k-variable model.
    """
    
    # Set random seed
    np.random.seed(random_seed)
    
    # Generate dataset with "true model" having exactly k variables
    print("Generating dataset...")
    X = np.random.randn(n_total, n_features)
    
    # Select which k variables are the true signal variables
    true_variables = np.random.choice(n_features, k, replace=False)
    true_variables = np.sort(true_variables)  # Keep sorted for easier tracking
    
    # Create true coefficients - only k variables have non-zero coefficients
    true_coefs = np.zeros(n_features)
    true_coefs[true_variables] = np.random.uniform(-3, 3, k)
    
    y = X @ true_coefs + noise_level * np.random.randn(n_total)
    
    print(f"True model uses variables: {true_variables}")
    print(f"True coefficients: {true_coefs[true_variables]}")
    
    # Initialize results storage
    results = {
        'sample_size': [],
        'iteration': [],
        'best_is_true_dgp': [],  # Is the empirically best model the true DGP?
    }
    
    # Add columns for each threshold
    for threshold in thresholds:
        results[f'count_{int(threshold*100)}pct'] = []
        results[f'dgp_in_rashomon_{int(threshold*100)}pct'] = []  # NEW: Is true DGP in Rashomon set?
    
    # Pre-compute all k-combinations and identify true model index
    all_combinations = list(combinations(range(n_features), k))
    n_combinations = len(all_combinations)
    
    # Find index of true model in the list of all combinations
    true_model_tuple = tuple(true_variables)
    true_model_index = all_combinations.index(true_model_tuple)
    
    print(f"Total possible {k}-variable subsets: {n_combinations:,}")
    print(f"True model is combination #{true_model_index}")
    print(f"Testing thresholds: {[f'{t*100:.1f}%' for t in thresholds]}")
    
    for sample_size in sample_sizes:
        print(f"\nProcessing sample size: {sample_size}")
        
        for iteration in tqdm(range(n_iterations), desc=f"Sample size {sample_size}"):
            # Sample from full dataset
            indices = np.random.choice(len(X), sample_size, replace=False)
            X_sample = X[indices]
            y_sample = y[indices]
            
            # Evaluate all k-combinations
            all_rss = []
            best_rss = float('inf')
            best_model_index = -1
            
            for i, feature_combo in enumerate(all_combinations):
                X_subset = X_sample[:, list(feature_combo)]
                model = LinearRegression()
                model.fit(X_subset, y_sample)
                y_pred = model.predict(X_subset)
                rss = np.sum((y_sample - y_pred) ** 2)
                all_rss.append(rss)
                
                if rss < best_rss:
                    best_rss = rss
                    best_model_index = i
            
            all_rss = np.array(all_rss)
            
            # Check if best model is the true DGP
            best_is_dgp = (best_model_index == true_model_index)
            
            # Store basic results
            results['sample_size'].append(sample_size)
            results['iteration'].append(iteration)
            results['best_is_true_dgp'].append(best_is_dgp)
            
            # For each threshold, count models and check if DGP is in Rashomon set
            for threshold in thresholds:
                threshold_value = best_rss * (1 + threshold)
                rashomon_mask = all_rss <= threshold_value
                count = np.sum(rashomon_mask)
                dgp_in_rashomon = rashomon_mask[true_model_index]
                
                col_name = f'count_{int(threshold*100)}pct'
                dgp_col_name = f'dgp_in_rashomon_{int(threshold*100)}pct'
                
                results[col_name].append(count)
                results[dgp_col_name].append(dgp_in_rashomon)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Print summary statistics
    print("\n" + "="*50)
    print("SIMULATION SUMMARY")
    print("="*50)
    
    for sample_size in sample_sizes:
        subset = results_df[results_df['sample_size'] == sample_size]
        best_is_dgp_rate = subset['best_is_true_dgp'].mean()
        
        print(f"\nSample size {sample_size}:")
        print(f"  Best model is true DGP: {best_is_dgp_rate:.1%}")
        
        for threshold in thresholds:
            dgp_col = f'dgp_in_rashomon_{int(threshold*100)}pct'
            count_col = f'count_{int(threshold*100)}pct'
            
            dgp_in_rashomon_rate = subset[dgp_col].mean()
            avg_rashomon_size = subset[count_col].mean()
            
            print(f"  {threshold*100:.1f}% threshold:")
            print(f"    DGP in Rashomon set: {dgp_in_rashomon_rate:.1%}")
            print(f"    Avg Rashomon set size: {avg_rashomon_size:.1f}")
    
    if save_results:
        results_df.to_csv('rashomon_simulation_results.csv', index=False)
        print(f"\nResults saved to rashomon_simulation_results.csv")
    
    return results_df

def print_summary(results_df, thresholds):
    """Print summary statistics of the simulation results."""
    print("\nSummary Statistics:")
    print("-" * 60)
    
    for sample_size in sorted(results_df['sample_size'].unique()):
        subset = results_df[results_df['sample_size'] == sample_size]
        print(f"\nSample size: {sample_size}")
        
        for threshold in thresholds:
            col_name = f'count_{int(threshold*100)}pct'
            mean_val = subset[col_name].mean()
            std_val = subset[col_name].std()
            print(f"  Models within {threshold*100:.1f}% of best: "
                  f"{mean_val:.1f} ± {std_val:.1f}")

def quick_rashomon_demo():
    """
    Run a quick demonstration with smaller parameters for testing.
    """
    print("Running quick demonstration with reduced parameters...")
    
    # Use smaller parameters for quick demo
    results = run_rashomon_simulation(
        thresholds=[0.01, 0.02, 0.05],
        sample_sizes=[500, 1000],
        n_iterations=10,
        n_total=2000,
        n_features=10,
        n_signal_vars=5,
        k=3,
        save_results=False
    )
    
    return results



In [ ]:
# Example usage in notebook:
if __name__ == "__main__":
    # For full simulation (this will take a while!):
    results = run_rashomon_simulation(thresholds=[0.01, 0.05])
    
    # For quick test:
    # results = quick_rashomon_demo()

Generating dataset...
True model uses variables: [ 1 12 15 21 22]
True coefficients: [-2.14328123 -0.58517476 -2.462441   -2.05218342  1.49414326]
Total possible 5-variable subsets: 142,506
True model is combination #41446
Testing thresholds: ['1.0%', '5.0%']

Processing sample size: 50


Sample size 50:  71%|██████████████████▍       | 71/100 [45:22<17:19, 35.85s/it]

In [ ]:
results.to_csv('../results/bootstrap_sim_big.csv', index=False)

In [ ]:
results